In [1]:
import numpy as NP

from astroutils import nonmathops as NMO

from ClosureInvariants import graphUtils as GU
# from ClosureInvariants import scalarInvariants as SI
from ClosureInvariants import scalarInvariants_torch as SI
import torch

from IPython.core.debugger import set_trace

## Set up antennas and array info

In [ ]:
element_ids = ['O', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
nelements_total = len(element_ids)
element_locs = 1e3*NP.random.randn(nelements_total,2) # in lambda units
elements_subset = ['O', 'A', 'C', 'E', 'G']
elements_remove = ['B', 'D', 'F']
nelements_subset = len(elements_subset)
element_indices_subset = NMO.find_list_in_list(element_ids, elements_subset)
element_locs_subset = element_locs[element_indices_subset,:]
print(element_indices_subset)

## Determine antenna pairs

In [ ]:
element_pairs = [(element_ids[i], element_ids[j]) for i in range(len(element_ids)) for j in range(i+1,len(element_ids))]
npairs = len(element_pairs)
element_pairs_locs = NP.array([element_locs[i] - element_locs[j] for i in range(len(element_ids)) for j in range(i+1,len(element_ids))]) # in lambda units
element_pairs_subset = [(elements_subset[i], elements_subset[j]) for i in range(len(elements_subset)) for j in range(i+1,len(elements_subset))]
npairs_subset = len(element_pairs_subset)
element_pairs_indices = [(i,j) for i in range(len(element_ids)) for j in range(i+1,len(element_ids))]
element_pairs_subset_indices = [(element_indices_subset[i],element_indices_subset[j]) for i in range(len(elements_subset)) for j in range(i+1,len(elements_subset))]
indices_of_subset_pairs = NMO.find_list_in_list(list(map(str,element_pairs)), list(map(str,element_pairs_subset))) # Indices of subset element pairs in all element pairs
element_pairs_subset_locs = element_pairs_locs[indices_of_subset_pairs]
print(element_pairs)
print(element_pairs_indices)
print(element_pairs_subset)
print(element_pairs_subset_indices)
print(indices_of_subset_pairs)

## Determine the complete and independent set of triads

In [ ]:
baseid = 'O'
base_ind = element_ids.index(baseid)
base_ind_subset = elements_subset.index(baseid)
triads_indep = GU.generate_independent_triads(element_ids, baseid='O')
triads_subset_indep = GU.generate_independent_triads(elements_subset, baseid='O')
indices_subset_triads = NMO.find_list_in_list(list(map(str,triads_indep)), list(map(str,triads_subset_indep))) # Indices of subset triads in all triads
print(triads_indep)
print(triads_subset_indep)
print(indices_subset_triads)

## Generate mock cross-correlations

In [ ]:
nruns_shape = (16,12,10) # Shape of number of runs (for example, number of random realisations, timestamps, channels)
npol = 1
bl_axis = -1 # Baseline axis
xc_real_std = 1.0
xc_imag_stad = 1.0
randseed = None
rng = NP.random.default_rng(randseed)
xcorr = rng.normal(loc=0.0, scale=xc_real_std, size=nruns_shape+(npairs,)) + 1j * rng.normal(loc=0.0, scale=xc_real_std, size=nruns_shape+(npairs,)) # The last axis is the baseline axis
xcorr_subset = NP.take(xcorr, indices_of_subset_pairs, axis=bl_axis) # Take a subset of the baseline axes based on the elements subset
xcorr = torch.tensor(xcorr)
xcorr_subset = torch.tensor(xcorr_subset)
print(xcorr.shape)
print(xcorr_subset.shape)


## Set up triangular loop correlations

In [ ]:
corrs_lol = SI.corrs_list_on_loops(xcorr, element_pairs, triads_indep, bl_axis=bl_axis) # _lol stands for list of lists
corrs_lol_subset_method1 = SI.corrs_list_on_loops(xcorr, element_pairs, triads_subset_indep, bl_axis=bl_axis) # Method 1 from all correlations
corrs_lol_subset_method2 = SI.corrs_list_on_loops(xcorr_subset, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
print(len(corrs_lol))
print(len(corrs_lol_subset_method1))
print(len(corrs_lol_subset_method2))
# Verify that both methods of setup produce identical results
# print(torch.max(torch.tensor([torch.max(i) for i in (NP.abs(NP.array(corrs_lol_subset_method1) - NP.array(corrs_lol_subset_method2)).flatten())])))


## Compute advariants

In [ ]:
advariants = SI.advariants_multiple_loops(corrs_lol)
advariants_subset = SI.advariants_multiple_loops(corrs_lol_subset_method1)
print(advariants.shape)
print(advariants_subset.shape)

## Compute the invariants by removing the unknown common scaling factor

In [ ]:
cloinv = SI.invariants_from_advariants_method1(advariants, normaxis=-1, normwts=None, normpower=2)
cloinv_subset = SI.invariants_from_advariants_method1(advariants_subset, normaxis=-1, normwts=None, normpower=2)
print(cloinv.shape)
print(cloinv_subset.shape)

## Test the invariance of the closure invariants obtained by corrupting the correlations using element-based gains

## Generate mock copolar gains

In [ ]:
element_axis = -1 # Antenna axis
mean_gain_scale = 3.0 
element_gains = rng.normal(loc=1.0, scale=NP.sqrt(0.5)/mean_gain_scale, size=nruns_shape+(nelements_total,)).astype(NP.float64) + 1j * rng.normal(loc=1.0, scale=NP.sqrt(0.5)/mean_gain_scale, size=nruns_shape+(nelements_total,)).astype(NP.float64) # shape is (...,n_element)
element_gains_subset = NP.take(element_gains, element_indices_subset, axis=element_axis)
print(element_gains.shape)
print(element_gains_subset.shape)

## Corrupt the correlations

In [ ]:
prefactor_gains = NP.take(element_gains, NP.array(element_pairs_indices)[:,0], axis=element_axis) # A collection of g_a
postfactor_gains = NP.take(element_gains, NP.array(element_pairs_indices)[:,1], axis=element_axis) # A collection of g_b
prefactor_gains = torch.tensor(prefactor_gains)
postfactor_gains = torch.tensor(postfactor_gains)
xcorr_mod = SI.corrupt_visibilities(xcorr, prefactor_gains, postfactor_gains)

prefactor_gains_subset = NP.take(element_gains, NP.array(element_pairs_subset_indices)[:,0], axis=element_axis) # A collection of g_a
postfactor_gains_subset = NP.take(element_gains, NP.array(element_pairs_subset_indices)[:,1], axis=element_axis) # A collection of g_b
prefactor_gains_subset = torch.tensor(prefactor_gains_subset)
postfactor_gains_subset = torch.tensor(postfactor_gains_subset)
xcorr_subset_mod = SI.corrupt_visibilities(xcorr_subset, prefactor_gains_subset, postfactor_gains_subset)

print(xcorr_mod.shape)
print(xcorr_subset_mod.shape)

## Compute the closure invariants through advariants, and scale factor elimination

In [ ]:
corrs_mod_lol = SI.corrs_list_on_loops(xcorr_mod, element_pairs, triads_indep, bl_axis=bl_axis)
corrs_mod_lol_subset_method1 = SI.corrs_list_on_loops(xcorr_mod, element_pairs, triads_subset_indep, bl_axis=bl_axis) # Method 1 from all correlations
corrs_mod_lol_subset_method2 = SI.corrs_list_on_loops(xcorr_subset_mod, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
print(len(corrs_mod_lol))
print(len(corrs_mod_lol_subset_method1))
print(len(corrs_mod_lol_subset_method2))
# Verify that both methods of setup produce identical results
# print(torch.max(torch.tensor([torch.max(i) for i in (NP.abs(NP.array(corrs_lol_subset_method1) - NP.array(corrs_lol_subset_method2)).flatten())])))


In [ ]:
advariants_mod = SI.advariants_multiple_loops(corrs_mod_lol)
advariants_mod_subset = SI.advariants_multiple_loops(corrs_mod_lol_subset_method1)
print(advariants_mod.shape)
print(advariants_mod_subset.shape)

In [ ]:
cloinv_mod = SI.invariants_from_advariants_method1(advariants_mod, normaxis=-1, normwts=None, normpower=2)
cloinv_mod_subset = SI.invariants_from_advariants_method1(advariants_mod_subset, normaxis=-1, normwts=None, normpower=2)
print(cloinv_mod.shape)
print(cloinv_mod_subset.shape)

## Check for invariance between ideal and modified versions

In [ ]:
print(NP.allclose(cloinv, cloinv_mod))
print(NP.allclose(cloinv_subset, cloinv_mod_subset))

## Verify scale factors are as expected, namely absolute squared value of the copolar complex gain of the base vertex

In [15]:
scale_factor = NP.abs(element_gains)**2
scale_factor_subset = NP.abs(element_gains_subset)**2

In [ ]:
print(NP.allclose(scale_factor[...,[base_ind]], advariants_mod / advariants))
print(NP.allclose(scale_factor_subset[...,[base_ind_subset]], advariants_mod_subset / advariants_subset))

## Point source verification

### Generate random point-like object location, intensity, and visibilities

In [17]:
src_loc_complex = NP.random.uniform() * NP.exp(1j*NP.random.uniform(low=-NP.pi, high=NP.pi))
src_loc = NP.array([src_loc_complex.real, src_loc_complex.imag]).reshape(1,-1) # in (l,m) coordinates
src_loc_0 = NP.zeros((1,2), dtype=float) # at origin / phase centre
src_amp = NP.random.rand() # in Jy
src_vis = NP.sum(src_amp * NP.exp(2j*NP.pi * NP.dot(element_pairs_locs, src_loc.T)), axis=-1)
src_vis_subset = src_vis[indices_of_subset_pairs]
src_0_vis = NP.sum(src_amp * NP.exp(2j*NP.pi * NP.dot(element_pairs_locs, src_loc_0.T)), axis=-1)
src_0_vis_subset = src_0_vis[indices_of_subset_pairs]

src_vis = torch.tensor(src_vis)
src_vis_subset = torch.tensor(src_vis_subset)
src_0_vis = torch.tensor(src_0_vis)
src_0_vis_subset = torch.tensor(src_0_vis_subset)

In [ ]:
print(src_loc)
print(src_loc_0)
print(src_vis.shape)
print(src_vis_subset.shape)

### Generate advariants for the source visibilities

In [19]:
src_corrs_lol = SI.corrs_list_on_loops(src_vis, element_pairs, triads_indep, bl_axis=bl_axis) # _lol stands for list of lists
src_corrs_lol_subset_method2 = SI.corrs_list_on_loops(src_vis_subset, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
src_advariants = SI.advariants_multiple_loops(src_corrs_lol)
src_advariants_subset = SI.advariants_multiple_loops(src_corrs_lol_subset_method2)

src_0_corrs_lol = SI.corrs_list_on_loops(src_0_vis, element_pairs, triads_indep, bl_axis=bl_axis) # _lol stands for list of lists
src_0_corrs_lol_subset_method2 = SI.corrs_list_on_loops(src_0_vis_subset, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
src_0_advariants = SI.advariants_multiple_loops(src_0_corrs_lol)
src_0_advariants_subset = SI.advariants_multiple_loops(src_0_corrs_lol_subset_method2)


In [ ]:
print(src_advariants.shape)

In [21]:
src_cloinv_normpow2 = SI.invariants_from_advariants_method1(src_advariants, normaxis=-1, normwts=None, normpower=2)
src_cloinv_subset_normpow2 = SI.invariants_from_advariants_method1(src_advariants_subset, normaxis=-1, normwts=None, normpower=2)
src_cloinv_normpow1 = SI.invariants_from_advariants_method1(src_advariants, normaxis=-1, normwts=None, normpower=1)
src_cloinv_subset_normpow1 = SI.invariants_from_advariants_method1(src_advariants_subset, normaxis=-1, normwts=None, normpower=1)

src_0_cloinv_normpow2 = SI.invariants_from_advariants_method1(src_0_advariants, normaxis=-1, normwts=None, normpower=2)
src_0_cloinv_subset_normpow2 = SI.invariants_from_advariants_method1(src_0_advariants_subset, normaxis=-1, normwts=None, normpower=2)
src_0_cloinv_normpow1 = SI.invariants_from_advariants_method1(src_0_advariants, normaxis=-1, normwts=None, normpower=1)
src_0_cloinv_subset_normpow1 = SI.invariants_from_advariants_method1(src_0_advariants_subset, normaxis=-1, normwts=None, normpower=1)

### Verify invariance to source translation

In [ ]:
print(NP.allclose(src_cloinv_normpow2, src_0_cloinv_normpow2))
print(NP.allclose(src_cloinv_normpow1, src_0_cloinv_normpow1))

### Verify point source closure invariants with "max" normalisation of advariants

In [23]:
src_cloinv_maxnorm_normpow1 = SI.invariants_from_advariants_method1(src_advariants, normaxis=-1, normwts='max', normpower=1)
src_cloinv_subset_maxnorm_normpow1 = SI.invariants_from_advariants_method1(src_advariants_subset, normaxis=-1, normwts='max', normpower=1)

In [24]:
expected_src_advariant = NP.ones_like(src_advariants)
expected_src_cloinv_maxnorm_normpow1 = NP.concatenate([expected_src_advariant.real, expected_src_advariant.imag], axis=-1)

expected_src_advariant_subset = NP.ones_like(src_advariants_subset)
expected_src_cloinv_subset_maxnorm_normpow1 = NP.concatenate([expected_src_advariant_subset.real, expected_src_advariant_subset.imag], axis=-1)

In [ ]:
print(NP.allclose(src_cloinv_maxnorm_normpow1, expected_src_cloinv_maxnorm_normpow1))
print(NP.allclose(src_cloinv_subset_maxnorm_normpow1, expected_src_cloinv_subset_maxnorm_normpow1))

### Verify point source closure phases

In [ ]:
src_cphases = SI.closurePhases_from_advariants(src_advariants)
print(NP.allclose(src_cphases, NP.zeros(src_advariants.shape)))

## Generate independent quads

In [ ]:
quads_indep = GU.generate_independent_quads(element_ids)
quads_subset_indep = GU.generate_independent_quads(elements_subset)
print(len(quads_subset_indep))
print(quads_subset_indep)
print('----------')
print(len(quads_indep))
print(quads_indep)

## Set up quadrilateral loop correlations

In [ ]:
quad_corrs_lol = SI.corrs_list_on_loops(xcorr, element_pairs, quads_indep, bl_axis=bl_axis) # _lol stands for list of lists
quad_corrs_lol_subset_method1 = SI.corrs_list_on_loops(xcorr, element_pairs, quads_subset_indep, bl_axis=bl_axis) # Method 1 from all correlations
quad_corrs_lol_subset_method2 = SI.corrs_list_on_loops(xcorr_subset, element_pairs_subset, quads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
print(len(quad_corrs_lol))
print(len(quad_corrs_lol_subset_method1))
print(len(corrs_lol_subset_method2))
# Verify that both methods of setup produce identical results
# print(torch.max(torch.tensor([torch.max(i) for i in (NP.abs(NP.array(corrs_lol_subset_method1) - NP.array(corrs_lol_subset_method2)).flatten())])))

## Compute Covariants

In [ ]:
covariants = SI.covariants_multiple_loops(quad_corrs_lol)
covariants_subset = SI.covariants_multiple_loops(quad_corrs_lol_subset_method1)
print(covariants.shape)
print(covariants_subset.shape)

## Compute Closure Amplitudes from Covariants

In [ ]:
clAmp = SI.closureAmplitudes_from_covariants(covariants)
clAmp_subset = SI.closureAmplitudes_from_covariants(covariants_subset)
print(clAmp.shape)
print(clAmp_subset.shape)

## Compute closure amplitudes to gain corruptions

In [ ]:
quad_corrs_mod_lol = SI.corrs_list_on_loops(xcorr_mod, element_pairs, quads_indep, bl_axis=bl_axis)
quad_corrs_mod_lol_subset_method1 = SI.corrs_list_on_loops(xcorr_mod, element_pairs, quads_subset_indep, bl_axis=bl_axis) # Method 1 from all correlations
quad_corrs_mod_lol_subset_method2 = SI.corrs_list_on_loops(xcorr_subset_mod, element_pairs_subset, quads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
print(len(quad_corrs_mod_lol))
print(len(quad_corrs_mod_lol_subset_method1))
print(len(quad_corrs_mod_lol_subset_method2))
# Verify that both methods of setup produce identical results
# print(torch.max(torch.tensor([torch.max(i) for i in (NP.abs(NP.array(corrs_lol_subset_method1) - NP.array(corrs_lol_subset_method2)).flatten())])))


In [ ]:
covariants_mod = SI.covariants_multiple_loops(quad_corrs_mod_lol)
covariants_mod_subset = SI.covariants_multiple_loops(quad_corrs_mod_lol_subset_method1)
print(covariants_mod.shape)
print(covariants_mod_subset.shape)

In [ ]:
clAmp_mod = SI.closureAmplitudes_from_covariants(covariants_mod)
clAmp_mod_subset = SI.closureAmplitudes_from_covariants(covariants_mod_subset)
print(clAmp_mod.shape)
print(clAmp_mod_subset.shape)

## Check for invariance between ideal and modified versions

In [ ]:
print(NP.allclose(clAmp, clAmp_mod))
print(NP.allclose(clAmp_subset, clAmp_mod_subset))

## Point source verification

### Generate covariants for the source visibilities

In [35]:
src_quad_corrs_lol = SI.corrs_list_on_loops(src_vis, element_pairs, quads_indep, bl_axis=bl_axis) # _lol stands for list of lists
src_quad_corrs_lol_subset_method2 = SI.corrs_list_on_loops(src_vis_subset, element_pairs_subset, quads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
src_covariants = SI.covariants_multiple_loops(src_quad_corrs_lol)
src_covariants_subset = SI.covariants_multiple_loops(src_quad_corrs_lol_subset_method2)

src_0_quad_corrs_lol = SI.corrs_list_on_loops(src_0_vis, element_pairs, quads_indep, bl_axis=bl_axis) # _lol stands for list of lists
src_0_quad_corrs_lol_subset_method2 = SI.corrs_list_on_loops(src_0_vis_subset, element_pairs_subset, quads_subset_indep, bl_axis=bl_axis) # Method 2 from subset correlations
src_0_covariants = SI.covariants_multiple_loops(src_0_quad_corrs_lol)
src_0_covariants_subset = SI.covariants_multiple_loops(src_0_quad_corrs_lol_subset_method2)

### Verify invariance of covariants to source translation. If that is satisfied, closure amplitudes, which are the amplitudes of covariants will also be satisfied.

In [ ]:
print(NP.allclose(src_covariants, src_0_covariants))
print(NP.allclose(src_covariants_subset, src_0_covariants_subset))

### Verify invariance of closure amplitudes to source translation

In [ ]:
src_clAmp = SI.closureAmplitudes_from_covariants(src_covariants)
src_0_clAmp = SI.closureAmplitudes_from_covariants(src_0_covariants)
print(NP.allclose(src_clAmp, src_0_clAmp))

### Verify point source closure amplitudes

In [ ]:
print(NP.allclose(src_clAmp, NP.ones(src_covariants.shape)))